# QRT Challenge - Exploratory Data Analysis

**Goal**: Predict the sign of an allocation's next-day return (binary classification)

**Features**:
- `RET_1` to `RET_20`: 20-day history of allocation returns (RET_1 = yesterday)
- `SIGNED_VOLUME_1` to `SIGNED_VOLUME_20`: 20-day history of volume-weighted liquidity
- `MEDIAN_DAILY_TURNOVER`: Allocation's median daily turnover
- `GROUP`: Anonymized allocation group
- `ALLOCATION`: Allocation identifier

**Target**: `TARGET` - next day return (we predict the sign: 1 if positive, 0 if negative)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

In [ ]:
# Load sample data (faster for EDA)
X_train = pd.read_csv('Data/X_train_sample.csv')
y_train = pd.read_csv('Data/y_train_sample.csv')

# Merge for easier analysis
df = X_train.merge(y_train, on='ROW_ID')

print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

## 1. Basic Data Overview

In [ ]:
# Basic info
print("=" * 50)
print("DATA TYPES")
print("=" * 50)
print(df.dtypes)
print("\n")

print("=" * 50)
print("MISSING VALUES")
print("=" * 50)
missing = df.isnull().sum()
print(missing[missing > 0])
print(f"\nTotal missing: {df.isnull().sum().sum()}")

In [ ]:
df.head(10)

In [ ]:
# Summary statistics for return columns
ret_cols = [f'RET_{i}' for i in range(1, 21)]
df[ret_cols + ['TARGET']].describe()

## 2. Target Distribution

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Target distribution (continuous)
axes[0].hist(df['TARGET'], bins=100, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='red', linestyle='--', label='Zero')
axes[0].set_xlabel('TARGET (Next-day Return)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Target Returns')
axes[0].legend()

# Binary target (what we predict)
df['TARGET_SIGN'] = (df['TARGET'] > 0).astype(int)
target_counts = df['TARGET_SIGN'].value_counts()
axes[1].bar(['Negative (0)', 'Positive (1)'], [target_counts[0], target_counts[1]], 
            color=['salmon', 'lightgreen'], edgecolor='black')
axes[1].set_ylabel('Count')
axes[1].set_title(f'Target Sign Distribution\n(Positive: {target_counts[1]/len(df)*100:.1f}%)')

# Box plot of target by sign
axes[2].boxplot([df[df['TARGET'] < 0]['TARGET'], df[df['TARGET'] > 0]['TARGET']], 
                labels=['Negative', 'Positive'])
axes[2].set_ylabel('Return Value')
axes[2].set_title('Target Value Distribution by Sign')

plt.tight_layout()
plt.show()

print(f"Class balance: {target_counts[1]/len(df)*100:.2f}% positive")

## 3. Return Features Analysis

In [ ]:
# Distribution of returns across different lags
fig, axes = plt.subplots(2, 5, figsize=(18, 8))
axes = axes.flatten()

for i, col in enumerate([f'RET_{j}' for j in [1, 2, 3, 4, 5, 10, 15, 20]] + ['TARGET']):
    if i < len(axes):
        axes[i].hist(df[col].dropna(), bins=50, edgecolor='black', alpha=0.7)
        axes[i].axvline(0, color='red', linestyle='--', alpha=0.5)
        axes[i].set_title(f'{col}\nmean={df[col].mean():.5f}')
        axes[i].set_xlabel('Return')

# Hide the last empty subplot
axes[-1].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# How do return statistics change across lags?
ret_stats = pd.DataFrame({
    'lag': range(1, 21),
    'mean': [df[f'RET_{i}'].mean() for i in range(1, 21)],
    'std': [df[f'RET_{i}'].std() for i in range(1, 21)],
    'skew': [df[f'RET_{i}'].skew() for i in range(1, 21)],
})

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(ret_stats['lag'], ret_stats['mean'], marker='o')
axes[0].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Lag (days ago)')
axes[0].set_ylabel('Mean Return')
axes[0].set_title('Mean Return by Lag')

axes[1].plot(ret_stats['lag'], ret_stats['std'], marker='o', color='orange')
axes[1].set_xlabel('Lag (days ago)')
axes[1].set_ylabel('Std Return')
axes[1].set_title('Return Volatility by Lag')

axes[2].plot(ret_stats['lag'], ret_stats['skew'], marker='o', color='green')
axes[2].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[2].set_xlabel('Lag (days ago)')
axes[2].set_ylabel('Skewness')
axes[2].set_title('Return Skewness by Lag')

plt.tight_layout()
plt.show()

## 4. Correlation Analysis

In [ ]:
# Correlation of each feature with TARGET
ret_cols = [f'RET_{i}' for i in range(1, 21)]
vol_cols = [f'SIGNED_VOLUME_{i}' for i in range(1, 21)]

ret_corr = df[ret_cols + ['TARGET']].corr()['TARGET'].drop('TARGET')
vol_corr = df[vol_cols + ['TARGET']].corr()['TARGET'].drop('TARGET')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Return correlations
axes[0].bar(range(1, 21), ret_corr.values, color='steelblue', edgecolor='black')
axes[0].axhline(0, color='red', linestyle='--')
axes[0].set_xlabel('Lag (days ago)')
axes[0].set_ylabel('Correlation with TARGET')
axes[0].set_title('RET_i Correlation with Target')
axes[0].set_xticks(range(1, 21))

# Volume correlations
axes[1].bar(range(1, 21), vol_corr.values, color='darkorange', edgecolor='black')
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_xlabel('Lag (days ago)')
axes[1].set_ylabel('Correlation with TARGET')
axes[1].set_title('SIGNED_VOLUME_i Correlation with Target')
axes[1].set_xticks(range(1, 21))

plt.tight_layout()
plt.show()

print("Top 5 RET correlations with TARGET:")
print(ret_corr.abs().sort_values(ascending=False).head())

In [ ]:
# Correlation heatmap for return features
plt.figure(figsize=(14, 10))
corr_matrix = df[ret_cols + ['TARGET']].corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix: Return Features + Target')
plt.tight_layout()
plt.show()

## 5. Group Analysis

In [ ]:
# How many groups? How are samples distributed?
print(f"Number of unique GROUPs: {df['GROUP'].nunique()}")
print(f"Number of unique ALLOCATIONs: {df['ALLOCATION'].nunique()}")
print(f"Number of unique timestamps: {df['TS'].nunique()}")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Samples per group
group_counts = df['GROUP'].value_counts().sort_index()
axes[0].bar(group_counts.index, group_counts.values, edgecolor='black')
axes[0].set_xlabel('GROUP')
axes[0].set_ylabel('Number of Samples')
axes[0].set_title('Sample Distribution by GROUP')

# Target mean by group
group_target = df.groupby('GROUP')['TARGET'].agg(['mean', 'std'])
axes[1].bar(group_target.index, group_target['mean'], yerr=group_target['std']/10, 
            edgecolor='black', capsize=3)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_xlabel('GROUP')
axes[1].set_ylabel('Mean Target Return')
axes[1].set_title('Mean Target by GROUP (error bars = std/10)')

plt.tight_layout()
plt.show()

In [ ]:
# Win rate (positive return %) by group
group_win_rate = df.groupby('GROUP')['TARGET_SIGN'].mean() * 100

plt.figure(figsize=(10, 5))
bars = plt.bar(group_win_rate.index, group_win_rate.values, edgecolor='black')
plt.axhline(50, color='red', linestyle='--', label='50% baseline')
plt.xlabel('GROUP')
plt.ylabel('Win Rate (%)')
plt.title('Positive Return Rate by GROUP')
plt.legend()

# Color bars based on win rate
for bar, rate in zip(bars, group_win_rate.values):
    bar.set_color('lightgreen' if rate > 50 else 'salmon')

plt.tight_layout()
plt.show()

print("Win rate by GROUP:")
print(group_win_rate.round(2))

## 6. Turnover Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Turnover distribution
axes[0].hist(df['MEDIAN_DAILY_TURNOVER'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Median Daily Turnover')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Turnover')

# Turnover vs Target (scatter)
sample_idx = np.random.choice(len(df), min(5000, len(df)), replace=False)
axes[1].scatter(df.iloc[sample_idx]['MEDIAN_DAILY_TURNOVER'], 
                df.iloc[sample_idx]['TARGET'], alpha=0.3, s=10)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_xlabel('Median Daily Turnover')
axes[1].set_ylabel('Target Return')
axes[1].set_title('Turnover vs Target')

# Turnover by group
df.boxplot(column='MEDIAN_DAILY_TURNOVER', by='GROUP', ax=axes[2])
axes[2].set_xlabel('GROUP')
axes[2].set_ylabel('Median Daily Turnover')
axes[2].set_title('Turnover Distribution by GROUP')
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.show()

# Correlation
print(f"Correlation between TURNOVER and TARGET: {df['MEDIAN_DAILY_TURNOVER'].corr(df['TARGET']):.4f}")

## 7. RET_1 Deep Dive (Most Important Feature)

In [ ]:
# RET_1 appears to be the most predictive feature - let's analyze it
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# RET_1 vs TARGET scatter
sample_idx = np.random.choice(len(df), min(5000, len(df)), replace=False)
axes[0].scatter(df.iloc[sample_idx]['RET_1'], df.iloc[sample_idx]['TARGET'], 
                alpha=0.3, s=10)
axes[0].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[0].axvline(0, color='red', linestyle='--', alpha=0.5)
axes[0].set_xlabel('RET_1 (Yesterday Return)')
axes[0].set_ylabel('TARGET (Today Return)')
axes[0].set_title(f'RET_1 vs TARGET\nCorr: {df["RET_1"].corr(df["TARGET"]):.4f}')

# Win rate by RET_1 quintile
df['RET_1_quintile'] = pd.qcut(df['RET_1'], 5, labels=['Q1(Low)', 'Q2', 'Q3', 'Q4', 'Q5(High)'])
quintile_win_rate = df.groupby('RET_1_quintile')['TARGET_SIGN'].mean() * 100
axes[1].bar(quintile_win_rate.index, quintile_win_rate.values, edgecolor='black')
axes[1].axhline(50, color='red', linestyle='--')
axes[1].set_xlabel('RET_1 Quintile')
axes[1].set_ylabel('Win Rate (%)')
axes[1].set_title('Win Rate by RET_1 Quintile')

# Mean target by RET_1 quintile
quintile_mean = df.groupby('RET_1_quintile')['TARGET'].mean() * 100  # in bps
colors = ['salmon' if x < 0 else 'lightgreen' for x in quintile_mean.values]
axes[2].bar(quintile_mean.index, quintile_mean.values, color=colors, edgecolor='black')
axes[2].axhline(0, color='red', linestyle='--')
axes[2].set_xlabel('RET_1 Quintile')
axes[2].set_ylabel('Mean Target Return (bps)')
axes[2].set_title('Mean Target by RET_1 Quintile')

plt.tight_layout()
plt.show()

# Clean up
df.drop('RET_1_quintile', axis=1, inplace=True)

In [ ]:
# Is there momentum or mean reversion?
# Momentum: RET_1 positive -> TARGET positive (same sign)
# Mean reversion: RET_1 positive -> TARGET negative (opposite sign)

df['RET_1_SIGN'] = (df['RET_1'] > 0).astype(int)

cross_tab = pd.crosstab(df['RET_1_SIGN'], df['TARGET_SIGN'], normalize='index') * 100
cross_tab.index = ['RET_1 < 0', 'RET_1 > 0']
cross_tab.columns = ['TARGET < 0', 'TARGET > 0']

print("Conditional probabilities (%):\n")
print(cross_tab.round(2))
print("\n")

# Interpretation
momentum_evidence = cross_tab.loc['RET_1 > 0', 'TARGET > 0'] > cross_tab.loc['RET_1 < 0', 'TARGET > 0']
print(f"Evidence of MOMENTUM: {momentum_evidence}")
print(f"If RET_1 > 0: {cross_tab.loc['RET_1 > 0', 'TARGET > 0']:.1f}% chance TARGET > 0")
print(f"If RET_1 < 0: {cross_tab.loc['RET_1 < 0', 'TARGET > 0']:.1f}% chance TARGET > 0")

## 8. Volume Features Analysis

In [ ]:
vol_cols = [f'SIGNED_VOLUME_{i}' for i in range(1, 21)]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Volume distribution for recent days
for col in ['SIGNED_VOLUME_1', 'SIGNED_VOLUME_5', 'SIGNED_VOLUME_10', 'SIGNED_VOLUME_20']:
    axes[0].hist(df[col].dropna(), bins=50, alpha=0.5, label=col)
axes[0].set_xlabel('Signed Volume')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Signed Volume')
axes[0].legend()

# Volume stats over time
vol_means = [df[f'SIGNED_VOLUME_{i}'].mean() for i in range(1, 21)]
vol_stds = [df[f'SIGNED_VOLUME_{i}'].std() for i in range(1, 21)]

axes[1].errorbar(range(1, 21), vol_means, yerr=np.array(vol_stds)/10, 
                 marker='o', capsize=3)
axes[1].axhline(0, color='red', linestyle='--')
axes[1].set_xlabel('Lag (days ago)')
axes[1].set_ylabel('Mean Signed Volume')
axes[1].set_title('Signed Volume by Lag (error = std/10)')

plt.tight_layout()
plt.show()

## 9. Feature Engineering Ideas

In [ ]:
# Test some potential engineered features
df_fe = df.copy()

# Rolling statistics
ret_cols = [f'RET_{i}' for i in range(1, 21)]
df_fe['RET_MEAN_5'] = df_fe[[f'RET_{i}' for i in range(1, 6)]].mean(axis=1)
df_fe['RET_MEAN_20'] = df_fe[ret_cols].mean(axis=1)
df_fe['RET_STD_5'] = df_fe[[f'RET_{i}' for i in range(1, 6)]].std(axis=1)
df_fe['RET_STD_20'] = df_fe[ret_cols].std(axis=1)

# Cumulative return
df_fe['RET_CUM_5'] = df_fe[[f'RET_{i}' for i in range(1, 6)]].sum(axis=1)
df_fe['RET_CUM_20'] = df_fe[ret_cols].sum(axis=1)

# Momentum indicators
df_fe['RET_POSITIVE_COUNT_5'] = (df_fe[[f'RET_{i}' for i in range(1, 6)]] > 0).sum(axis=1)
df_fe['RET_POSITIVE_COUNT_20'] = (df_fe[ret_cols] > 0).sum(axis=1)

# Recent vs old
df_fe['RET_RECENT_VS_OLD'] = df_fe['RET_MEAN_5'] - df_fe[[f'RET_{i}' for i in range(16, 21)]].mean(axis=1)

# Check correlations with target
new_features = ['RET_MEAN_5', 'RET_MEAN_20', 'RET_STD_5', 'RET_STD_20', 
                'RET_CUM_5', 'RET_CUM_20', 'RET_POSITIVE_COUNT_5', 
                'RET_POSITIVE_COUNT_20', 'RET_RECENT_VS_OLD']

correlations = df_fe[new_features + ['TARGET']].corr()['TARGET'].drop('TARGET')

plt.figure(figsize=(10, 5))
colors = ['lightgreen' if x > 0 else 'salmon' for x in correlations.values]
plt.barh(correlations.index, correlations.values, color=colors, edgecolor='black')
plt.axvline(0, color='black', linestyle='-')
plt.xlabel('Correlation with TARGET')
plt.title('Engineered Features: Correlation with Target')
plt.tight_layout()
plt.show()

print("\nEngineered feature correlations:")
print(correlations.sort_values(ascending=False).round(4))

## 10. Key Takeaways

### Summary of findings:

1. **Target Balance**: Check if classes are balanced (~50/50 split)

2. **RET_1 is dominant**: Yesterday's return is by far the most predictive feature

3. **Momentum vs Mean Reversion**: Analyze the conditional probabilities to determine the regime

4. **Group differences**: Some groups may have systematically different win rates

5. **Feature engineering**: Rolling means, cumulative returns, and momentum indicators may help

6. **Volume features**: Generally weaker predictors than return features

### Next steps:
- Build baseline models (Logistic Regression, Random Forest, XGBoost)
- Feature selection based on importance
- Cross-validation strategy (careful with time-series aspects)
- Ensemble methods